# Project: cleanning - [candy data]
### this notebook is for educational purpose . in this notebook we will do some cleanning

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">wrangling</a></li>
  
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> i will go through candy data 2017 , this data has :
- Internal_ID
- Q1-Going Out?: Are you actually going trick or treating yourself? "has values `Yes or No`"
- Q2-Gender: It has four different options, `Femal ,Male ,Other ,I'd rather not say`
- Q3-Age: Numerical field
- Q4-Country: Text Field, but users have written their own version of the names. Example, for America, there are entries such as USA, us, US, America so we should consider that while cleanning
- Q5-State/Province: Text Field, but users have written their own version of the names. Same as the country data.
- Q6-Joy Or Despair: All kinds of chocolate bars are the questions with three distinct options to choose from (Joy, Meh, Despair).
- Q7-Joy Other: Text Field to write items not included above that give you JOY. Lots of missing values.
- Q8-Despair Other: Text Field ti write items not included above that give you DESPAIR. Lots of missing values.
- Q9-Other Comments: Text Field. Lots of missing values.
- Q10-Dress: Binary field. Missing values present
- Unnamed: 113
- Q11-Day: Binary Answer Field. Missing values present


# 1- import packages and load data

In [8]:
# pip install matplotlib
import pandas as pd 
import numpy as nd
# import matplotlib.pyplot as plt
pd.options.display.max_columns

df = pd.read_excel('candyhierarchy2017.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'candyhierarchy2017.xlsx'

<a id='wrangling'></a>
## Data Wrangling



### General Properties

# 2- show some general properties for the data


In [ ]:
df.shape
# df.describe()

(2460, 120)

In [ ]:
df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we don’t know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,...,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | Reese’s Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,...,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,...,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,JOY,MEH,JOY,NaN,JOY,JOY,DESPAIR,NaN,MEH,DESPAIR,JOY,

# 3- write summary about what you get form the above functions
> #### tips : this summary can be about : if there is any missing values , if there a wrong data type , if there a wrong range of values , ....

The dataset appears to have : 
- many NaN value
- collapsible values (us,USA,usa..etc)
- irrelevant and useless rows

# cleaning

### 4- drop all the useless columns and  rows with too much NaN , 
> tip : useless columns can be like `Internal ID`and can be also columns with too much NaN 
- rename columns for easy access

In [ ]:
df = df.drop(['Internal ID','Click Coordinates (x, y)','Unnamed: 113'], axis =1)
df.head()

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we don’t know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,Q6 | Hershey’s Milk Chocolate,...,Q6 | Pencils,Q6 | Pixy Stix,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | Reese’s Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,...,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,Sunday,NaN,1.0,NaN,NaN
2,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,MEH,...,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,Sunday,NaN,1.0,NaN,NaN
4,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,...,DESPAIR,JOY,DESPAIR,JOY,JOY,MEH,JOY,NaN,JOY,JOY,DESPAIR,NaN,MEH,DESPAIR,JOY,JOY,M

In [ ]:
names_map = {'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)': 'Anonymous globs ',
             'Q6 | Candy that is clearly just the stuff given out for free at restaurants':'cheap candy',
             'Q6 | Sandwich-sized bags filled with BooBerry Crunch':"Booberry bags",
             'Q6 | Real Housewives of Orange County Season 9 Blue-Ray':'Real Housewives',
             'Q6 | Any full-sized candy bar':'full candy bar',
             'Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein':'corn syrup',
             'Q6 | Those odd marshmallow circus peanut things	':'marshmallow circus peanuts',  
             'Q6 | Bonkers (the candy)':'Bonkers candy',
             'Q6 | Bonkers (the board game)':'Bonkers board game',	
             'Q6 | Cash, or other forms of legal tender' : 'legal tender',
             'Q6 | Chick-o-Sticks (we don’t know what that is)':'chick sticks candy',
             'Q5: STATE, PROVINCE, COUNTY, ETC':'AREA'
             }
df = df.rename(columns = names_map,)

In [ ]:
#cleaning useless redundancy
columns = list(df.columns)
new_col = []
for col in columns:
  if ':' in col :
    new_col.append(col.split(':')[1].strip())
  elif '|' in col :
    new_col.append(col.split('|')[1].strip())
  else :
    new_col.append(col)
df.columns = new_col
df.head()

,GOING OUT?,GENDER,AGE,COUNTRY,area,100 Grand Bar,Anonymous globs,full candy bar,Black Jacks,Bonkers candy,Bonkers board game,Bottle Caps,Box'o'Raisins,Broken glow stick,Butterfinger,Cadbury Creme Eggs,Candy Corn,cheap candy,Caramellos,legal tender,Chardonnay,chick sticks candy,Chiclets,Coffee Crisp,Creepy Religious comics/Chick Tracts,Dental paraphenalia,Dots,Dove Bars,Fuzzy Peaches,Generic Brand Acetaminophen,Glow sticks,Goo Goo Clusters,Good N' Plenty,Gum from baseball cards,Gummy Bears straight up,Hard Candy,Healthy Fruit,Heath Bar,Hershey's Dark Chocolate,Hershey’s Milk Chocolate,...,Pencils,Pixy Stix,Real Housewives,Reese’s Peanut Butter Cups,Reese's Pieces,Reggie Jackson Bar,Rolos,Booberry bags,Skittles,Smarties (American),Smarties (Commonwealth),Snickers,Sourpatch Kids (i.e. abominations of nature),Spotted Dick,Starburst,Sweet Tarts,Swedish Fish,Sweetums (a friend to diabetes),Take 5,Tic Tacs,Those odd marshmallow circus peanut things,Three Musketeers,Tolberone something or other,Trail Mix,Twix,corn syrup,Vicodin,Whatchamacallit Bars,White Bread,Whole Wheat anything,York Peppermint Patties,JOY OTHER,DESPAIR OTHER,OTHER COMMENTS,DRESS,DAY,MEDIA [Daily Dish],MEDIA [Science],MEDIA [ESPN],MEDIA [Yahoo]
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,...,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,Sunday,NaN,1.0,NaN,NaN
2,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,MEH,...,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,Sunday,NaN,1.0,NaN,NaN
4,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,...,DESPAIR,JOY,DESPAIR,JOY,JOY,MEH,JOY,NaN,JOY,JOY,DESPAIR,NaN,MEH,DESPAIR,JOY,JOY,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,NaN,NaN,NaN,White and gold,Friday,NaN,1.0,NaN,NaN


### 5 - clean `going out` column 
> tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['GOING OUT?'].isna().sum()
#since 110 resemble less than 5% of the total dataset,dropping is considered
df = df.dropna(subset=['GOING OUT?'])

### 6 - clean ` gender` column 
> tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['GENDER'].unique()
df['GENDER'].isna().sum()

14

In [ ]:
# imputing NaN values in variable like "Gender" to "prefer not to say" would be an option.
df['GENDER'].fillna("I'd rather not say",inplace =True)
df['GENDER'].isna().any()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


False

### 7 - clean ` age` column 
> tip : note the type of age column , and NaN values
 - try to use `to_numeric` 
 - tip : you should choose which best value to put in place of these NaNs ( mean, median, ....etc)

In [ ]:
df['AGE'].dtype
df['AGE'].unique()

array([44, 40, 23, nan, 53, 33, 43, 56, 64, 37, 59, 48, 54, 36, 45, 49,
       25, 34, 35, 38, 58, 50, 47, 16, 52, 63, 41, 27, 61, 46, 42, 62, 29,
       39, 32, 31, 28, 69, 67, 22, 26, 51, 70, 24, 18, 19, 'Old enough',
       57, 60, 66, 12, 30, 'Many', 65, 55, 72, '?', 21, 11, 'no', 9, 68,
       20, 6, 10, 71, 90, 13, '45-55', 312, 99, 7, 'hahahahaha', 88, 39.4,
       74, 102, 'older than dirt', 17, 15, 8, 75, '5u', 'Enough',
       'See question 2', '24-50', 14, 'Over 50', 100, 76, 1000,
       'sixty-nine', '46 Halloweens.', 'ancient', 77, 'OLD', 'old', 73,
       70.5, 1, 'MY NAME JEFF', 4, '59 on the day after Halloween',
       'old enough', 'your mom',
       'I can remember when Java was a cool new language', '60+'],
      dtype=object)

In [ ]:
df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce', downcast='integer')
# df['AGE'] = df['AGE'].astype(int)
df['AGE'].dtype

dtype('float64')

In [ ]:
# df = df.loc[df['AGE'].astype(int) <= 120 ]
# df['AGE']

In [ ]:
df['AGE'].fillna(df['AGE'].mean(),inplace = True)


Series([], Name: AGE, dtype: float64)


### 8 - clean ` country` column
> tip : note that country column has country names with different writing ways 
  - use fuzzywuzzy to deal with this problem
  - you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
pip install fuzzywuzzy

In [ ]:
df['COUNTRY'].isna().sum()
df = df.dropna(subset=['COUNTRY'])

In [ ]:
df['COUNTRY'] = df['COUNTRY'].astype(str)
df['COUNTRY'] = df['COUNTRY'].str.strip().str.upper()
# df['COUNTRY'].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from fuzzywuzzy import process

#since there are many of variants.Extract function is used to detect strings similarities
possible_countries = pd.Series(['USA','UK','UNITED STATES OF AMERICA',])

for country in possible_countries:
  matches = process.extract(country , df['COUNTRY'], limit =df.shape[0])
  for candidate_match in matches:
    # print(candidate_match)
    if candidate_match[1] >= 80 :
      df.loc[df['COUNTRY'] == candidate_match[0], 'COUNTRY'] = country
df['COUNTRY'].unique()


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


array(['USA', 'CANADA', 'MURICA', 'UNITED STATES OF AMERICA', 'UK', 'UAE',
       'ENGLAND', 'MEXICO', 'U.S.A.', '35', 'FRANCE', 'FINLAND',
       'NORTH CAROLINA', 'NETHERLANDS', 'GERMANY', 'EUROPE', 'EARTH',
       'COSTA RICA', '46', 'AUSTRALIA', 'INSANITY LATELY', 'GREECE',
       'USA? HARD TO TELL ANYMORE..', 'PITTSBURGH', '45', '32', 'CAN',
       'CANAE', 'TRUMPISTAN', 'IRELAND', 'KOREA', 'CALIFORNIA', 'JAPAN',
       'SOUTH AFRICA', 'ICELAND', 'SCOTLAND', 'DENMARK', 'SWITZERLAND',
       'AHEM....AMERCA', 'UD', 'NEW JERSEY', 'INDONESIA',
       'THE NETHERLANDS', 'ENDLAND', 'ATLANTIS', 'MURRIKA', 'ALASKA',
       'SOVIET CANUCKISTAN', 'SINGAPORE', 'CHINA', 'TAIWAN', 'HONG KONG',
       'SPAIN', 'SWEDEN', 'NARNIA', 'U S A', '1',
       'SUBSCRIBE TO DM4UZ3 ON YOUTUBE', "I DON'T KNOW ANYMORE",
       'FEAR AND LOATHING'], dtype=object)

In [ ]:
#collapse the remaning manually
map = {'UNITED STATES OF AMERICA':'USA'
       , 'U.S.A.':'USA', 'NORTH CAROLINA':'USA'
       , 'USA? HARD TO TELL ANYMORE..':'USA'
       , 'NEW JERSEY':'USA','NETHERLANDS':'EUROPE'
       , 'U S A':'USA' , 'CALIFORNIA':'USA'
       ,'AHEM....AMERCA' :'USA', 'ENGLAND':'UK',
       'SCOTLAND':'UK','SWEDEN':'EUROPE','UK':'EUROPE',
       'SPAIN':'EUROPE','FRANCE':'EUROPE','FINLAND':'EUROPE',
       'DENMARK':'EUROPE','FINLAND':'EUROPE','GERMANY':'EUROPE',
       'ICELAND':'EUROPE','ENDLAND':'EUROPE','IRELAND':'EUROPE',
       'GREECE':'EUROPE','THE NETHERLANDS':'EUROPE','SWITZERLAND':'EUROPE',
       'KOREA':'ASIA', 'CHINA':'ASIA','TAIWAN':'ASIA',
       'SOVIET CANUCKISTAN':'ASIA','JAPAN':'ASIA','HONG KONG':'ASIA',
       'TRUMPISTAN':'ASIA'
       ,}
df['COUNTRY'] = df['COUNTRY'].replace(map)
df['COUNTRY'] = df['COUNTRY'].loc[~ df['COUNTRY'].str.isdigit()]
df['COUNTRY'].unique()

array(['USA', 'CANADA', 'MURICA', 'EUROPE', 'UAE', 'UK', 'MEXICO', nan,
       'EARTH', 'COSTA RICA', 'AUSTRALIA', 'INSANITY LATELY',
       'PITTSBURGH', 'CAN', 'CANAE', 'ASIA', 'SOUTH AFRICA', 'UD',
       'INDONESIA', 'ATLANTIS', 'MURRIKA', 'ALASKA', 'SINGAPORE',
       'NARNIA', 'SUBSCRIBE TO DM4UZ3 ON YOUTUBE', "I DON'T KNOW ANYMORE",
       'FEAR AND LOATHING'], dtype=object)

### 9 - clean ` area` column
> tip : area column has the same problem as country columns , but looking at all it's unique values may give us another way to deal with it rather than try to fix it

In [ ]:
df['area'].unique()
#I have no idea.

array(['NM', 'or', 'exton pa', 'Colorado ', 'ontario', 'Ontario', 'Wa',
       'Quebec', 'NY', 'California', 'Texas', 'NEW YORK', 'CO', 'IN',
       'MN', 'aberdeenshire', 'Alberta', 'Wisconsin, 53705', 'RI', 'NC',
       'Scotland', 'minnesota', 'Michigan', 'Florida', 'New Jersey', 'VA',
       'Colorado', 'Los Angeles, California', 'CA', 'New York ',
       'Chicago, Illinois', 'Nj', 'Wisconsin', 'Minnesota ', nan,
       'London', 'mn', 'TEXAS', 'Illinois', 'District of Columbia',
       'Washington', 'OR', 'Montana', 'New York', 'massachusetts',
       'Abu Dhabi', 'TX', 'Arizona', 'yukon', 'Pennsylvania ', 'MD',
       'Ohio', 'Alberta ', 'OXFORDSHIRE', 'DC', 'Iowa', 'Mn', 'TN', 'BC',
       'Maryland', 'MI', 'Wisconsin ', 'alaska', 'Ny',
       'Colorado, Jefferson county ', 'Louisiana', 'pa',
       'CO, Larimer County', 'Nebraska ', 'Oregon', 'UT', 'ny',
       'Pennsylvania', 'CDMX', 'north carolina', 'Tennessee ', 'Texas ',
       'Knox county, Tennessee', 'New Mexico', 'main

### 10 - clean ` Q6` columns
>- tip : Q6 has all kinds of chocolate bars and questions with three distinct options to choose from (Joy, Meh, Despair).
- so all the column that has Q6 is a candy name , so you should go through all these column and you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

### 11 - clean ` dress` column
>tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['DRESS'].mode()

0    White and gold
dtype: object

In [ ]:
df['DRESS'].fillna('White and gold',inplace = True)

### 12 - clean ` day` column
>tip : you should choose which best value to put in place of these NaNs ( mode, unknown,other ....etc)

In [ ]:
df['DAY'].fillna('either',inplace=True)

False

### 13 - check
> after finish cleaning you should make sure your data is clean and that :
- there is no wrong data type
- there is no NaN values 
- all the column are cleaing and ready for the next step `EDA`

In [ ]:
df.isna().any()

GOING OUT?            False
GENDER                False
AGE                   False
COUNTRY                True
area                   True
                      ...  
DAY                   False
MEDIA [Daily Dish]     True
MEDIA [Science]        True
MEDIA [ESPN]           True
MEDIA [Yahoo]          True
Length: 117, dtype: bool

### 14 - write a summary about all the cleaning steps 

many cleaning techinques have been used according to the nature of variable and 
NaN count.
for example:
1. droping is considered in low nan-values columns
2. imputing missing values to mode or mean according to type of data wether it a scall or catagory